In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats # libreria estadistica de Scipy
from sklearn.feature_selection import RFE # RFE es para seleccionar modelos
from sklearn.model_selection import train_test_split # para dividir la base en train y test
from sklearn import linear_model # para modelo lineal
from sklearn.metrics import mean_squared_error, r2_score # para sacar las metricas
import statsmodels.api as sm  ## Parte estadistica
from statsmodels.sandbox.regression.predstd import wls_prediction_std  ## Parte estadistica



In [5]:
ruta_archivo ="../Datos/regresion/student-mat.csv"

In [6]:
datos = pd.read_csv(ruta_archivo, delimiter=';', na_values="-")
datos


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [7]:
hay_na = datos.isna().any().any()
print("¿Hay valores NA en el DataFrame?:", hay_na)

¿Hay valores NA en el DataFrame?: False


In [12]:
columnas_seleccionadas = ['studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']

In [13]:
Data =datos[columnas_seleccionadas]
Data

,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,2,0,4,3,4,1,1,3,6,5,6,6
1,2,0,5,3,3,1,1,3,4,5,5,6
2,2,3,4,3,2,2,3,3,10,7,8,10
3,3,0,3,2,2,1,1,5,2,15,14,15
4,2,0,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...
390,2,2,5,5,4,4,5,4,11,9,9,9
391,1,0,2,4,5,3,4,2,3,14,16,16
392,1,3,5,5,3,3,3,3,3,10,8,7
393,1,0,4,4,1,3,4,5,0,11,12,10


In [15]:
Data.describe()

,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [14]:
# Calcular la matriz de correlación
correlacion = Data.corr()

# Mostrar la matriz de correlación
print(correlacion)

           studytime  failures    famrel  freetime     goout      Dalc  \
studytime   1.000000 -0.173563  0.039731 -0.143198 -0.063904 -0.196019   
failures   -0.173563  1.000000 -0.044337  0.091987  0.124561  0.136047   
famrel      0.039731 -0.044337  1.000000  0.150701  0.064568 -0.077594   
freetime   -0.143198  0.091987  0.150701  1.000000  0.285019  0.209001   
goout      -0.063904  0.124561  0.064568  0.285019  1.000000  0.266994   
Dalc       -0.196019  0.136047 -0.077594  0.209001  0.266994  1.000000   
Walc       -0.253785  0.141962 -0.113397  0.147822  0.420386  0.647544   
health     -0.075616  0.065827  0.094056  0.075733 -0.009577  0.077180   
absences   -0.062700  0.063726 -0.044354 -0.058078  0.044302  0.111908   
G1          0.160612 -0.354718  0.022168  0.012613 -0.149104 -0.094159   
G2          0.135880 -0.355896 -0.018281 -0.013777 -0.162250 -0.064120   
G3          0.097820 -0.360415  0.051363  0.011307 -0.132791 -0.054660   

               Walc    health  absenc

Análisis de las correlaciones:
 Correlación con G1:

- studytime: 0.1606 (relación positiva leve)
- failures: -0.3547 (relación negativa moderada)
- goout: -0.1491 (relación negativa leve)



Correlación con G2:

- studytime: 0.1358 (relación positiva leve)
- failures: -0.3558 (relación negativa moderada)
- goout: -0.1622 (relación negativa leve)


 Correlación con G3:

- studytime: 0.0978 (relación positiva débil)
- failures: -0.3604 (relación negativa moderada)
- goout: -0.1327 (relación negativa leve)




Variables con correlación más fuerte:


Basándonos en el análisis de correlación, parece que las variables más prometedoras para predecir las notas son:

- failures (correlación negativa moderada con todas las notas).
- studytime (aunque su correlación es leve, tiene una relación positiva con las tres notas).
- goout (tiene una correlación negativa leve con las tres notas, lo que podría influir en el desempeño académico).

## Modelo de regresión lineal múltiple

In [49]:
# Variables predictoras basadas en el análisis de correlación
X = datos[['failures', 'studytime', 'goout', 'Dalc']]

# Variables objetivo (notas que queremos predecir)
Y = datos[['G1', 'G2', 'G3']]

In [50]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [51]:
from sklearn.linear_model import LinearRegression

# Crear el modelo de regresión lineal
modelo = LinearRegression()

# Ajustar el modelo con los datos de entrenamiento
modelo.fit(X_train, Y_train)

LinearRegression()

In [52]:
Y_pred = modelo.predict(X_test)

In [53]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluar el modelo con R²
print("Coeficiente de Determinación (R²):", r2_score(Y_test, Y_pred))

# Evaluar el modelo con MSE
print("Error Cuadrático Medio (MSE):", mean_squared_error(Y_test, Y_pred))

Coeficiente de Determinación (R²): 0.131400279666291
Error Cuadrático Medio (MSE): 14.387777678227337
